In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from scipy.special import softmax

In [ ]:
# Fonction d'activation ReLU
def relu(x):
    return np.maximum(0, x)

# Dérivée de ReLU
def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [ ]:
# Couche entièrement connectée (Dense Layer)
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * 0.1
        self.biases = np.random.randn(1, output_size) * 0.1
    
    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.biases
    
    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(self.input.T, output_gradient)
        input_gradient = np.dot(output_gradient, self.weights.T)
        self.weights -= learning_rate * weights_gradient
        self.biases -= learning_rate * np.sum(output_gradient, axis=0, keepdims=True)
        return input_gradient

In [ ]:
# Fonction de perte (Cross-Entropy)
def cross_entropy_loss(predictions, labels):
    return -np.mean(np.sum(labels * np.log(predictions), axis=1))

# Dérivée de la fonction de perte
def cross_entropy_loss_derivative(predictions, labels):
    return predictions - labels

In [ ]:
# Chargement et prétraitement des données MNIST
def load_mnist_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train / 255.0  # Normalisation entre 0 et 1
    x_test = x_test / 255.0

    # Convertir les labels en encodage one-hot
    y_train_one_hot = np.eye(10)[y_train]
    y_test_one_hot = np.eye(10)[y_test]
    
    return (x_train, y_train_one_hot), (x_test, y_test_one_hot)

In [ ]:
# Réseau de neurones avec votre couche convolutionnelle
class SimpleCNN:
    def __init__(self):
        self.conv = Convolutional(input_shape=(1, 28, 28), kernel_size=3, depth=8)  # Convolution
        self.dense = Dense(1352, 10)  # Couche dense pour 10 classes (28-3+1)² * 8 = 1352
        
    def forward(self, x):
        x = x.reshape((-1, 1, 28, 28))  # Redimensionner pour correspondre à la forme d'entrée de la convolution
        x = self.conv.forward(x[0])  # Propagation avant de la couche convolutionnelle
        x = relu(x)  # Activation ReLU
        x = x.flatten()  # Mise à plat
        x = self.dense.forward(x)  # Propagation avant de la couche dense
        return softmax(x)  # Fonction softmax pour la sortie (probabilités)

    def backward(self, output_gradient, learning_rate):
        gradient = self.dense.backward(output_gradient, learning_rate)
        gradient = gradient.reshape((self.conv.output_shape))  # Reshape to match convolution output
        gradient = relu_derivative(self.conv.output) * gradient  # Appliquer dérivée de ReLU
        self.conv.backward(gradient, learning_rate)

In [ ]:
# Entraînement du modèle sur MNIST
def train(model, x_train, y_train, epochs, learning_rate):
    for epoch in range(epochs):
        loss = 0
        for i in range(len(x_train)):
            # Propagation avant
            output = model.forward(x_train[i])

            # Calcul de la perte
            loss += cross_entropy_loss(output, y_train[i])

            # Propagation arrière
            output_gradient = cross_entropy_loss_derivative(output, y_train[i])
            model.backward(output_gradient, learning_rate)

        # Afficher la perte moyenne pour chaque epoch
        loss /= len(x_train)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')


In [ ]:
# Test du modèle
def test(model, x_test, y_test):
    correct_predictions = 0
    for i in range(len(x_test)):
        output = model.forward(x_test[i])
        if np.argmax(output) == np.argmax(y_test[i]):
            correct_predictions += 1
    accuracy = correct_predictions / len(x_test)
    print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Charger les données MNIST
(x_train, y_train), (x_test, y_test) = load_mnist_data()
# Créer le modèle CNN
model = SimpleCNN()
# Entraîner le modèle
train(model, x_train, y_train, epochs=3, learning_rate=0.01)
# Tester le modèle
test(model, x_test, y_test)